In [138]:
# Fix: corrected index usage in time-weighted objective function and flow accumulation terms

import gurobipy as gp
from gurobipy import GRB

# Time and demand settings
tau = 0.25
T = 60.0
N = int(T / tau)
D_EV = 36.0
D_ICV = 36.0

# Flow parameters
V = 900.0
W = 450.0
K = 0.12
Q_a = K * V * W / (V + W)

# Link lengths
length = {1: 0.0, 2: 900.0, 3: 1800.0, 4: 900.0, 5: 3600.0, 6: 900.0, 7: 0.0}
travelV = {a: int((length[a]/V)/tau + 1e-9) for a in length} #dividiamo per tau così invece che avere il tempo di percorrrenza abbiamo quanti step ci vogliono per percorrere
travelW = {a: int((length[a]/W)/tau + 1e-9) for a in length}

# Battery parameters
omega = 0.33
epsilon = 1.8e-4
B_max = 0.5
B0 = 0.35

# Budget constraint
I_budget = 2800.0

# Network topology
E = [1, 2, 3, 4, 5, 6, 7]
E_R, E_S, E_A = [1], [7], [2, 3, 4, 5, 6]
pred = {1: [], 2: [1], 3: [1], 4: [2], 5: [2], 6: [3, 4], 7: [5, 6]}
succ = {1: [2, 3], 2: [4, 5], 3: [6], 4: [6], 5: [7], 6: [7], 7: []}

# EV paths
paths = {0: [2, 4, 6], 1: [2, 5], 2: [3, 6]}
delta = {(a, p): int(a in paths[p]) for p in paths for a in E_A}
count_paths = {a: sum(delta[(a, p)] for p in paths) for a in E_A}

model = gp.Model("Dynamic_WCL")
model.setParam("OutputFlag", 0)

x = model.addVars(E_A, vtype=GRB.BINARY, name="x")
y = model.addVars(len(paths), vtype=GRB.BINARY, name="y")
B = {(a, p): model.addVar(ub=B_max, name=f"B_{a}_{p}") for p in paths for a in paths[p]}

classes = ['EV', 'ICV']
u, v, n, f = {}, {}, {}, {}
for m in classes:
    for a in E:
        for i in range(0, N+1):
            u[(m, a, i)] = model.addVar(lb=0.0)
            v[(m, a, i)] = model.addVar(lb=0.0)
            n[(m, a, i)] = model.addVar(lb=0.0)
        for b in pred[a]:
            for i in range(0, N+1):
                f[(m, b, a, i)] = model.addVar(lb=0.0)

alpha = model.addVars(E_A, classes, range(1, N+1), lb=0.0, ub=1.0, name="alpha")
for a in E_A:
    for i in range(1, N+1):
        model.addConstr(gp.quicksum(alpha[a, m, i] for m in classes) == 1.0)

# Corrected time-weighted objective function (Eq. 1)
model.setObjective(gp.quicksum((T - (i - 1)*tau) *f[(m, b, 7, i)] for m in classes for b in pred[7] for i in range(1, N+1)), GRB.MAXIMIZE)
#model.setObjective(4*gp.quicksum(n[m,a,i] for m in classes for a in E[:-1] for i in range(0, N+1)), GRB.MINIMIZE)

model.addConstr(gp.quicksum(length[a] * x[a] for a in E_A) <= I_budget)

#upper bound per v e u
for a in E:
    for i in range(0,N+1):
        model.addConstr(gp.quicksum(v[(m,a,i)] for m in classes) <= Q_a)
        


M = 1e3
#constraints energetiche 4-6
for p, link_list in paths.items():
    for idx, a in enumerate(link_list):
        prevB = B0 if idx == 0 else B[(link_list[idx - 1], p)]
        recharge = omega * (length[a] / V)
        consume = epsilon * length[a]
        model.addConstr(B[(a, p)] <= prevB - consume + recharge * x[a] + M * (1 - y[p]))
        model.addConstr(B[(a, p)] >= prevB - consume + recharge * x[a] - M * (1 - y[p]))
        model.addConstr(B[(a,p)]>= M* (y[p]-1))


#15 e 18
for i in range(0, N+1):
    model.addConstr(u[('EV', 1, i)] == D_EV)
    model.addConstr(u[('ICV', 1, i)] == D_ICV)
    model.addConstr(v[('EV', 7, i)] == 0)
    model.addConstr(v[('ICV', 7, i)] == 0)
#16 e 17
for m in classes:
    for a in E:
        for i in range(0, N+1):
            if a not in E_R:
                model.addConstr(u[(m, a, i)] == gp.quicksum(f[(m, b, a, i)] for b in pred[a]))
            if a not in E_S:
                model.addConstr(v[(m, a, i)] == gp.quicksum(f[(m, a, b, i)] for b in succ[a]))
#13 e 14
for m in classes:
    for a in E_A:
        for i in range(0, N+1):
            inflow = gp.quicksum(u[(m, a, k)] for k in range(0, i+1))  # 12
            outflow = gp.quicksum(v[(m, a, k)] for k in range(0, i+1))
            model.addConstr(n[(m, a, i)] == inflow - outflow)
            #if a in E_A:
            if i >= travelV[a]:
                model.addConstr(n[(m, a, i)] >= gp.quicksum(u[(m, a, k)] for k in range(i - travelV[a] + 1, i+1)))
             #  if i >= travelW[a]:
                   #model.addConstr(n[(m, a, i)] + gp.quicksum(v[(m, a, k)] for k in range(i - travelW[a] + 1, i+1)) <= K * length[a] * alpha[a, m, i])
for a in E_A:
    for m in classes:
        for i in range(1, N+1):
            model.addConstr(n[(m, a, i)] <= K * length[a] * alpha[a, m, i], name=f"cap_consistency_{a}_{m}_{i}")

#20-23
for a in E_A:
    for i in range(0, N+1):
        inflow_s = gp.quicksum(u[(m, a, k)] for m in classes for k in range(0, i))
        outflow_s = gp.quicksum(v[(m, a, k)] for m in classes for k in range(0, i - travelW[a] + 1)) if i > travelW[a] else 0
        supply = K * length[a] + outflow_s - inflow_s
        model.addConstr(gp.quicksum(u[(m, a, i)] for m in classes) <= Q_a)
        model.addConstr(gp.quicksum(u[(m, a, i)] for m in classes) <= supply)

        inflow_d = gp.quicksum(v[(m, a, k)] for m in classes for k in range(0, i))
        outflow_d = gp.quicksum(u[(m, a, k)] for m in classes for k in range(0, i - travelV[a] + 1)) if i > travelV[a] else 0
        demand = outflow_d - inflow_d
        model.addConstr(gp.quicksum(v[(m, a, i)] for m in classes) <= Q_a)
        model.addConstr(gp.quicksum(v[(m, a, i)] for m in classes) <= demand)
#19
for m in classes:
    for a in E_A:
        for i in range(0, N+1):
            inflow = gp.quicksum(f[(m, b, a, i)] for b in pred[a])
            if m == 'EV':
                model.addConstr(inflow <= Q_a * gp.quicksum(delta[(a, p)] * y[p] for p in paths))
            else:
                model.addConstr(inflow <= Q_a * count_paths[a])

model.optimize()


In [139]:
# === Output: WCL installation and EV path choices ===
if model.status == GRB.OPTIMAL or model.status == GRB.SUBOPTIMAL:
    print(f"🎯 Valore funzione obiettivo (outflow pesato): {model.ObjVal:.2f}")
    for p in paths:
         for a in paths[p]:
             if B[(a,p)].x > 1e-4:
                 print(f"energia B[({a},{p})]={B[(a,p)]}")
    
    print("🔌 Link selezionati per l'installazione delle WCL:")
    for a in E_A:
        if x[a].x > 0.5:
            print(f"  - Link {a} (lunghezza: {length[a]} m)")

    print("🚗 Percorsi selezionati per EV:")
    for p in paths:
        if y[p].x > 0.5:
            print(f"  - Path {p}: 1 -> " + " -> ".join(str(a) for a in paths[p]) + " -> 7")
else:
    print("⚠️ Nessuna soluzione ottima trovata.")



🎯 Valore funzione obiettivo (outflow pesato): 116550.00
energia B[(2,0)]=<gurobi.Var B_2_0 (value 0.18799999999998818)>
energia B[(4,0)]=<gurobi.Var B_4_0 (value 0.025999999999953616)>
energia B[(6,0)]=<gurobi.Var B_6_0 (value 0.19400000000018713)>
energia B[(3,2)]=<gurobi.Var B_3_2 (value 0.025999999999953616)>
energia B[(6,2)]=<gurobi.Var B_6_2 (value 0.19400000000018713)>
🔌 Link selezionati per l'installazione delle WCL:
  - Link 6 (lunghezza: 900.0 m)
🚗 Percorsi selezionati per EV:
  - Path 0: 1 -> 2 -> 4 -> 6 -> 7
  - Path 2: 1 -> 3 -> 6 -> 7


In [140]:
print("\n📈 Flusso in entrata e uscita (solo se > 0):")
for a in E:
    for i in range(0,N+1):
        for m in classes:
            if u[m, a, i].X > 1e-3 or v[m, a, i].X > 1e-3:
                print(f"  t={i:.2f}, link={a}, tipo={m}: u = {u[m,a,i].X:.2f}, v = {v[m,a,i].X:.2f}")
                
print("\n↔️ Flusso f[a, b, m, t] (solo se > 0):")
for a in E:
    for b in succ[a]:
        for m in classes:
            for i in range(N):
                if f[m,a,b,i].X > 1e-3:
                    print(f"  f[{m},{a},{b},{i:.2f}] = {f[m,a,b,i].X:.2f}")


📈 Flusso in entrata e uscita (solo se > 0):
  t=0.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=0.00, link=1, tipo=ICV: u = 36.00, v = 36.00
  t=1.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=1.00, link=1, tipo=ICV: u = 36.00, v = 36.00
  t=2.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=2.00, link=1, tipo=ICV: u = 36.00, v = 36.00
  t=3.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=3.00, link=1, tipo=ICV: u = 36.00, v = 36.00
  t=4.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=4.00, link=1, tipo=ICV: u = 36.00, v = 36.00
  t=5.00, link=1, tipo=EV: u = 36.00, v = 36.00
  t=5.00, link=1, tipo=ICV: u = 36.00, v = 0.00
  t=6.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=6.00, link=1, tipo=ICV: u = 36.00, v = 36.00
  t=7.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=7.00, link=1, tipo=ICV: u = 36.00, v = 0.00
  t=8.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=8.00, link=1, tipo=ICV: u = 36.00, v = 0.00
  t=9.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=9.00, link=1, tipo=ICV: u = 36.00, v = 0.0

In [141]:
print("\n--- u[(m,a,i)] max ---")
for m in classes:
    for a in E:
        max_u = max(u[(m, a, i)].X for i in range(N+1))
        if max_u > 0:
            print(f"Max u[{m},{a}] = {max_u:.2f}")

print("\n--- v[(m,a,i)] max ---")
for m in classes:
    for a in E:
        max_v = max(v[(m, a, i)].X for i in range(N+1))
        if max_v > 0:
            print(f"Max v[{m},{a}] = {max_v:.2f}")



--- u[(m,a,i)] max ---
Max u[EV,1] = 36.00
Max u[EV,3] = 36.00
Max u[EV,6] = 36.00
Max u[EV,7] = 36.00
Max u[ICV,1] = 36.00
Max u[ICV,2] = 36.00
Max u[ICV,3] = 36.00
Max u[ICV,4] = 36.00
Max u[ICV,5] = 36.00
Max u[ICV,6] = 36.00
Max u[ICV,7] = 72.00

--- v[(m,a,i)] max ---
Max v[EV,1] = 36.00
Max v[EV,3] = 36.00
Max v[EV,6] = 36.00
Max v[ICV,1] = 36.00
Max v[ICV,2] = 36.00
Max v[ICV,3] = 36.00
Max v[ICV,5] = 36.00
Max v[ICV,6] = 36.00


In [142]:
print("\n--- n[(m,a,i)] ---")
for m in classes:
    for a in E:
        max_n = max(n[(m, a, i)].X for i in range(N+1))
        if max_n > 0:
            print(f"Max n[{m},{a}] = {max_n:.2f}")



--- n[(m,a,i)] ---
Max n[EV,3] = 108.00
Max n[EV,6] = 108.00
Max n[ICV,2] = 108.00
Max n[ICV,3] = 108.00
Max n[ICV,4] = 36.00
Max n[ICV,5] = 216.00
Max n[ICV,6] = 108.00


In [143]:
for m in classes:
    for a in E:
        if length[a] > 0:
            max_n = max(n[(m, a, i)].X for i in range(N+1))
            print(f"Max n[{m},{a}] = {max_n:.2f} | cap = {K * length[a]:.2f}")


Max n[EV,2] = 0.00 | cap = 108.00
Max n[EV,3] = 108.00 | cap = 216.00
Max n[EV,4] = 0.00 | cap = 108.00
Max n[EV,5] = 0.00 | cap = 432.00
Max n[EV,6] = 108.00 | cap = 108.00
Max n[ICV,2] = 108.00 | cap = 108.00
Max n[ICV,3] = 108.00 | cap = 216.00
Max n[ICV,4] = 36.00 | cap = 108.00
Max n[ICV,5] = 216.00 | cap = 432.00
Max n[ICV,6] = 108.00 | cap = 108.00
